<a href="https://colab.research.google.com/github/Rushikesh1137/IMDB-Movie-Reviews-Sentiment-Analysis-Using-LSTM/blob/main/IMDB_Reviews_sentiments.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install kaggle

In [2]:
import os
import json #to read file from the kaggle
##import tensorflow as tf


from zipfile import ZipFile
import pandas as pd
from sklearn.model_selection import train_test_split

from tensorflow.keras.models import Sequential

from tensorflow.keras.layers import Embedding, LSTM,Dense

from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

In [ ]:
# from google.colab import drive
# drive.mount('/content/drive')

In [3]:
kaggle_dictionary = json.load(open("kaggle.json"))

In [4]:
 os.environ["KAGGLE_USERNAME"] = kaggle_dictionary["username"]
 os.environ["KAGGLE_KEY"] = kaggle_dictionary["key"]

Loading dataset as a zip file

In [5]:
!kaggle datasets download -d lakshmi25npathi/imdb-dataset-of-50k-movie-reviews

Dataset URL: https://www.kaggle.com/datasets/lakshmi25npathi/imdb-dataset-of-50k-movie-reviews
License(s): other
  0% 0.00/25.7M [00:00<?, ?B/s]
100% 25.7M/25.7M [00:00<00:00, 1.24GB/s]


In [6]:
!ls

imdb-dataset-of-50k-movie-reviews.zip  kaggle.json  sample_data


In [7]:
#unzip the data file
with ZipFile("imdb-dataset-of-50k-movie-reviews.zip","r") as zip_ref:
  zip_ref.extractall()

In [8]:
!ls

'IMDB Dataset.csv'			 kaggle.json
 imdb-dataset-of-50k-movie-reviews.zip	 sample_data


Loading the **Dataset**

In [9]:
data = pd.read_csv("IMDB Dataset.csv") ##or else can provide the path

In [10]:
data.head()


,review,sentiment
0,One of the other reviewers has mentioned that ...,positive
1,A wonderful little production. <br /><br />The...,positive
2,I thought this was a wonderful way to spend ti...,positive
3,Basically there's a family where a little boy ...,negative
4,"Petter Mattei's ""Love in the Time of Money"" is...",positive


In [11]:
data["sentiment"].value_counts() #no of unique values in the col

,count
sentiment,
positive,25000
negative,25000


encode the *positive as 1* and *negative as 0*

In [12]:
data.replace({"positive":1,"negative":0},inplace=True)

/tmp/ipython-input-2739597079.py:1: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  data.replace({"positive":1,"negative":0},inplace=True)


In [13]:
train_data, test_data = train_test_split(data,test_size=0.2, random_state = 42)

In [14]:
print(train_data.shape)
print(test_data.shape)

(40000, 2)
(10000, 2)


**DATA PREprocessing**

In [15]:
tokenizer= Tokenizer(num_words=5000)
tokenizer.fit_on_texts(train_data["review"])
X_train = pad_sequences(tokenizer.texts_to_sequences(train_data["review"]),maxlen = 200)
X_test = pad_sequences(tokenizer.texts_to_sequences(test_data["review"]),maxlen = 200)

In [16]:
print(X_test)

[[   0    0    0 ...  995  719  155]
 [  12  162   59 ...  380    7    7]
 [   0    0    0 ...   50 1088   96]
 ...
 [   0    0    0 ...  125  200 3241]
 [   0    0    0 ... 1066    1 2305]
 [   0    0    0 ...    1  332   27]]


In [17]:
print(X_train)

[[1935    1 1200 ...  205  351 3856]
 [   3 1651  595 ...   89  103    9]
 [   0    0    0 ...    2  710   62]
 ...
 [   0    0    0 ... 1641    2  603]
 [   0    0    0 ...  245  103  125]
 [   0    0    0 ...   70   73 2062]]


In [18]:
Y_train = train_data["sentiment"]
Y_test = test_data["sentiment"]

In [19]:
print(Y_test)

33553    1
9427     1
199      0
12447    1
39489    0
        ..
28567    0
25079    1
18707    1
15200    0
5857     1
Name: sentiment, Length: 10000, dtype: int64


In [20]:
print(Y_train)

39087    0
30893    0
45278    1
16398    0
13653    0
        ..
11284    1
44732    1
38158    0
860      1
15795    1
Name: sentiment, Length: 40000, dtype: int64


# Building LSTM Model

In [21]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dense

# Let's assume your sequences are padded to length 100
sequence_length = 200  # must match your actual input data length

model = Sequential()
model.add(Embedding(input_dim=5000, output_dim=128, input_length=sequence_length))
model.add(LSTM(128, dropout=0.2, recurrent_dropout=0.2))
model.add(Dense(1, activation="sigmoid"))

# Now call summary — model is built



/usr/local/lib/python3.11/dist-packages/keras/src/layers/core/embedding.py:90: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


In [22]:
model.build(input_shape=(None, 200))  # (batch_size, sequence_length)
model.summary()


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ embedding (Embedding)           │ (None, 200, 128)       │       640,000 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm (LSTM)                     │ (None, 128)            │       131,584 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 1)              │           129 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 771,713 (2.94 MB)

 Trainable params: 771,713 (2.94 MB)

 Non-trainable params: 0 (0.00 B)

In [23]:
model.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

# Training the Model


In [24]:
model.fit(X_train, Y_train, batch_size=64, epochs=5, validation_split=0.2)

Epoch 1/5
500/500 ━━━━━━━━━━━━━━━━━━━━ 204s 395ms/step - accuracy: 0.7075 - loss: 0.5438 - val_accuracy: 0.8406 - val_loss: 0.3685
Epoch 2/5
500/500 ━━━━━━━━━━━━━━━━━━━━ 196s 389ms/step - accuracy: 0.8255 - loss: 0.4069 - val_accuracy: 0.8386 - val_loss: 0.3734
Epoch 3/5
500/500 ━━━━━━━━━━━━━━━━━━━━ 204s 392ms/step - accuracy: 0.8681 - loss: 0.3223 - val_accuracy: 0.8518 - val_loss: 0.3444
Epoch 4/5
500/500 ━━━━━━━━━━━━━━━━━━━━ 200s 389ms/step - accuracy: 0.8750 - loss: 0.3056 - val_accuracy: 0.8611 - val_loss: 0.3308
Epoch 5/5
500/500 ━━━━━━━━━━━━━━━━━━━━ 199s 384ms/step - accuracy: 0.8990 - loss: 0.2577 - val_accuracy: 0.8529 - val_loss: 0.3444


In [25]:
model.save("model.h5")

In [26]:
import joblib
joblib.dump(tokenizer,"tokenizer.pkl")

['tokenizer.pkl']

# Model Evaulation

In [27]:
loss, accuracy = model.evaluate(X_test, Y_test)
print(f"Test Loss: {loss}, Test Accuracy: {accuracy}")

313/313 ━━━━━━━━━━━━━━━━━━━━ 35s 111ms/step - accuracy: 0.8549 - loss: 0.3333
Test Loss: 0.33187857270240784, Test Accuracy: 0.8600000143051147


**Building a predictive System**

In [28]:
def predict_sentiment(review):
  sequence = tokenizer.texts_to_sequences([review])
  padded_sequence = pad_sequences(sequence, maxlen=200)
  prediction = model.predict(padded_sequence)
  sentiment = "positive" if prediction[0][0] > 0.5 else "negative"
  return sentiment

In [29]:
new_review = "This movie was below average! I didn't enjoyed it."
predicted_sentiment = predict_sentiment(new_review)
print(f"Predicted Sentiment: {predicted_sentiment}")

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 427ms/step
Predicted Sentiment: negative


In [30]:
new_review = "However, if you an forget all the nonsense, it's actually a very lovable and undeniably entertaining romp that delivers as plenty of thrills, and unintentionally, plenty of laughs.<br /><br />You've got to love John Lithgows sneery evilness, his tick every box band of baddies, and best of all, the permanently harassed and hapless 'turncoat' agent, Rex Linn as Travers.<br /><br />He may of been Henry in 'Portrait of a Serial Killer' but Michael Rooker is noteworthy for a cringe-worthy performance as Hal, he insists on constantly shrieking in painful disbelief at his captors 'that man never hurt anybody' And whilst he surely can't be, it really does look like Ralph Waite's Frank character is grinning as the girl plummets to her death.<br /><br />Mention too must go to former 'London's Burning' actor Craig Fairbrass as the Brit bad guy, who comes a cropper whilst using Hal as a Human Football, yes, you can't help enjoy that bit, Hal needed a good kicking.<br /><br />So forget your better judgement, who cares if 'that could never happen', lower your acting expectations, turn up the volume and enjoy! And if you're looking for Qaulen, he's the one wearing the helicopter."
predicted_sentiment = predict_sentiment(new_review)
print(f"Predicted Sentiment: {predicted_sentiment}")

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 130ms/step
Predicted Sentiment: positive
